In [27]:
import numpy as np
from scipy.stats import norm, chisquare
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation
import matplotlib.colors as colors
import matplotlib.ticker as tkr
import sys
from importlib import reload as rel
sys.path.insert(0, "../../python/")
sys.path.insert(0, "../")
from Constants import SI
import vsim_analysis as va
import vsim.load as load
import vsim.analyze as analyze
import h5py
import matplotlib.mlab as mlab
global kb, me
kb = SI.boltzmann; # J/K
me  = SI.elecMass; # kg
plt.style.use('notes')
#% matplotlib notebook

# Plotting Code

In [31]:
def alpha_colormap(cmap, cutoff, flip=True):
    N = cmap.N
    cmapt = cmap(np.arange(N))
    alpha = np.ones(N)
    if flip:
        temp = alpha[:int(cutoff*N)]
        M = len(temp)
        alpha[:int(cutoff*N)] = np.linspace(0, 1, M)
    else:
        alpha[int((1-cutoff)*N):] = 0.0
    cmapt[:, -1] = alpha
    cmapt = colors.ListedColormap(cmapt)
    return cmapt

In [32]:
def plot_gas_evolution(gas, pulse, t, LX, LY, savePath, aniName,save = False):
    '''
    Plots the gas density and laser pulse throughout the simulation

    Parameters:
    -----------
    gas : array_like
        data of the gas to be plotted
    pulse : array_like
        data of the laser pulse to be plotted
    t : array_like
        time at each dump used in the plot in fs
    LX : int
        longitudinal length of the simulation (um)
    LY : int 
        transverse length of the simulation (um)
    savePath : string
        path to save the animation to
    aniName : string
        name of the animation
    save : boolean, optional
        set True if you want to save the animation
    '''
    gas = np.squeeze(gas)
    fig = plt.figure(figsize = (5,3.5), dpi = 150)
    ax  = fig.gca()
    ext = np.array([-LX/2, LX/2, -LY/2, LY/2])


    # y-intensity (for pulse)
    pulse = pulse[:,:,:,1]**2
    pulse = pulse.transpose()
    cmap_p = alpha_colormap(plt.cm.get_cmap('viridis'), .01 * np.amax(pulse))
    im = plt.imshow(gas[0].transpose(), extent = ext, cmap = 'jet', \
                    aspect = 'auto', animated = True)
    im2 = plt.imshow(pulse[:,:,0], extent = ext,\
                     aspect = 'auto', animated = True, alpha = .5, \
                     cmap = cmap_p, \
                     vmin = np.amin(pulse), vmax = np.amax(pulse))

    plt.xlabel(r'x ($\mu$m)')
    plt.ylabel(r'y ($\mu$m)')
    cbar = plt.colorbar(mappable = im)
    cbar.set_label(r'm$^{-3}$')
    def updatefig(i, gas, pulse):
        im.set_data(gas[i].transpose())
        im2.set_data(pulse[:,:,i])
        plt.title('t = %.2f' %t[i] + ' (fs) \n Neutral gas density')
    ani = animation.FuncAnimation(fig, updatefig, frames = len(gas), \
                    fargs = (gas, pulse), interval = 1000)
    if save:
        ani.save(savePath + aniName)

In [33]:
def plot_pulse(pulse, t, LX, LY, savePath, aniName, save = False):
    fig = plt.figure(figsize = (5,3.5), dpi = 150)
    ax  = fig.gca()
    ext = np.array([-LX/2, LX/2, -LY/2, LY/2])
    pulse = pulse[:,:,:,1]#**2
    pulse = pulse.transpose()
    im = plt.imshow(pulse[:,:,0], extent = ext,\
                     aspect = 'auto', animated = True, \
                     vmin = np.amin(pulse), vmax = np.amax(pulse) )
    plt.xlabel(r'x ($\mu$m)')
    plt.ylabel(r'y ($\mu$m)')
    #cbar = plt.colorbar(mappable = im)
    #cbar.set_label(r'm$^{-3}$')
    def updatefig(i, pulse = pulse):
        im.set_array(pulse[:,:,i])
        plt.title('t = %.2f' %t[i] + ' (fs) \n Laser pulse')	
    ani = animation.FuncAnimation(fig, updatefig, frames = len(pulse[0,0,:]),\
                    interval = 1000)
    if save:
        ani.save(savePath + aniName)

In [34]:
def make_color(weights):
    # Make a color scale by particle weight
    minima = min(weights)
    maxima = max(weights)
    norm = colors.Normalize(vmin = minima, vmax = maxima, clip = True)
    mapper = cm.ScalarMappable(norm = norm, cmap = cm.jet)
    return mapper.to_rgba(weights)
def plot_phase_space(species, ts, t, i_start,savePath, aniName, save = False):
    # Set dimensions for plot
    fig = plt.figure(figsize = (5,3.5), dpi = 150)
    ax  = fig.gca()
    ax.set_xlabel(r'$\gamma v_x$ (1e6 m/s)') 
    ax.set_ylabel(r'$\gamma v_y$ (1e6 m/s)')
    scatt = ax.scatter([], [], s = .25, marker = 'o')
    ax.set_ylim([-8, 8])
    ax.set_xlim([-8, 8])
    xdata, ydata = [], []
    def update(i, species = species):
        #print(i)
        if i >= i_start:
            xdata = analyze.get_ux(species[i-i_start])
            ydata = analyze.get_uy(species[i-i_start])
            weights = analyze.get_weights(species[i-i_start])
            scatt.set_color(make_color(weights))
            scatt.set_offsets(np.c_[xdata*1e-6,ydata*1e-6])
        ax.set_title(' t = %.2f' % t[i] + ' (fs) \n' + ts)
    ani = animation.FuncAnimation(fig, update, frames = len(t), \
                                 interval = 1000)
    if save:
        ani.save(savePath + aniName)
     

In [35]:
def plot_v_dist(species, ts, t, i_start, savePath, aniName, save = False):
    fig = plt.figure(figsize = (5, 7), dpi = 150)
    ax1  = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    ax1.set_xlabel(r'$\gamma v_x (1e6 m/s)$')
    ax1.set_ylabel('Counts (A.U.)')
    ax2.set_xlabel(r'$\gamma v_y (1e6 m/s)$')
    ax2.set_ylabel('Counts (A.U.)')
    plt.subplots_adjust(hspace = 0.2)
    def update(i, species = species):
        if i >= i_start:
            ax1.clear()
            ax2.clear()
            ux = analyze.get_ux(species[i-i_start])
            uy = analyze.get_uy(species[i-i_start])
            weights = analyze.get_weights(species[i-i_start])

            
            nx, binsx, px = ax1.hist(ux, 200, weights = weights)#, normed = True)
            ny, binsy, py = ax2.hist(uy, 200, weights = weights)#, normed = True)
            
            xlocsx = ax1.get_xticks()
            ax1.set_xticklabels(xlocsx/1e6)
            
            xlocsy = ax2.get_xticks()
            ax2.set_xticklabels(xlocsy/1e6)
            
        ax1.set_title('t = %.2f' % t[i] + '(fs) \n' + ts)
        ax1.set_xlabel(r'$\gamma v_x (1e6 m/s)$')
        ax1.set_ylabel('Counts (A.U.)')
        ax2.set_xlabel(r'$\gamma v_y (1e6 m/s)$')
        ax2.set_ylabel('Counts (A.U.)')
    ani = animation.FuncAnimation(fig, update, frames = len(t), \
                                 interval = 1000)
    if save:
        ani.save(savePath + aniName)

In [36]:
def ionization_vis(gas, pulse, electrons, i_start,\
                   t, LX, LY, savePath, \
                   aniName, save = False):
    fig = plt.figure(figsize = (5, 10))
    xl = [-LX/2, LX]
    yl = [-LY/2, LY/2]
    ax1 = fig.add_subplot(211)
    ax1.xaxis.set_ticklabels([])
    ax1.set_xlim(xl)
    ax1.set_ylim(yl)
    ax1.set_ylabel(r'y ($\mu$m)')
    
    ax2 = fig.add_subplot(212)
    ax2.set_xlim(xl)
    ax2.set_ylim(yl)
    ax2.set_ylabel(r'y ($\mu$m)')
    ax2.set_xlabel(r'x ($\mu$m)')

    
    
    gas = np.squeeze(gas)
    ext = np.array([-LX/2, LX/2, -LY/2, LY/2])

    plt.subplots_adjust(hspace = 0.2)
    # y-intensity (for pulse)
    pulse = pulse[:,:,:,1]**2
    pulse = pulse.transpose()
    cmap_p = alpha_colormap(plt.cm.get_cmap('viridis'), .01 * np.amax(pulse))
    im = ax1.imshow(gas[0].transpose(), extent = ext, cmap = 'jet', \
                     aspect = 'auto', animated = True)
    im2 = ax1.imshow(pulse[:,:,0], extent = ext,\
                     aspect = 'auto', animated = True, alpha = .5, \
                     cmap = cmap_p, \
                     vmin = np.amin(pulse), vmax = np.amax(pulse))

    #cbar = plt.colorbar(mappable = im)
    #cbar.set_label(r'm$^{-3}$')
    scatt1 = ax2.scatter([], [], s = 0.25, marker = 'o')
    
    def update(i, gas = gas, pulse = pulse, i_start = i_start, \
                  electrons = electrons ):
        im.set_data(gas[i].transpose())
        im2.set_data(pulse[:,:,i])
        if i >= i_start:
            xdata_e = analyze.get_x(electrons[i-i_start])
            ydata_e = analyze.get_y(electrons[i-i_start])
            weights_e = analyze.get_weights(electrons[i-i_start])
            uy = analyze.get_uy(electrons[i-i_start])
            ux = analyze.get_ux(electrons[i-i_start])
            scatt1.set_color(make_color(weights_e))
            scatt1.set_offsets(np.c_[xdata_e*1e6 - 60, ydata_e*1e6])


        
        
    ani = animation.FuncAnimation(fig, update, frames = len(t), \
                                 interval = 1000)
    if save:
        ani.save(savePath + aniName)
ionization_vis(gas_data, edgeE_data, elec_data, 5,\
                   t, LX, LY, '/home/keenan/Desktop/', \
                   'test.mp4', save = True) 

# Load Data

In [24]:
simPath = '/home/keenan/Desktop/VSim_Test/Ionize_v02/08_28_Output/'
simName = 'laserIonize'
savePath = '/media/keenan/Data_Storage/Google Drive/CU/PWFA/Notes/08_2018/'
LX = 140;
LY = 100;

In [25]:
# Time info 
timestep = 130.62996926e-18;
nsteps   = 2553;
dump_per = 127; 
nfiles   = 22;

In [26]:
edgeE_data, gas_data, elec_data, Ar1_data, Ar2_data, attrs = \
                                            va.get_data(5,22,simPath, simName)

t = np.zeros(len(attrs))
for i in range(len(attrs)):
    t[i] = attrs[i]['time']*1e15

# Visualization

In [48]:
plot_pulse(edgeE_data, t, LX, LY, savePath, 'pulse_8_30.mp4', \
           save = True)

In [40]:
plot_gas_evolution(gas_data, edgeE_data, t, 140, 100, '/home/keenan/Desktop/', \
                                                  'test.mp4', save = True)

In [49]:
plot_phase_space(elec_data, 'Ionized electrons', t, 5 ,savePath,\
                 'electron_phase_8_30.mp4', save = True )

In [50]:
plot_v_dist(elec_data, 'Ionized electrons VDF ', \
            t, 5, savePath, 'Ionized_Electron_VDFs_8_30.mp4', save = True)

In [71]:
ionization_vis(gas_data, edgeE_data, elec_data, Ar1_data,elec_data, 5, t, LX, \
               LY, 'Ionization of Argon', '/home/keenan/Documents/', \
                   'test.mp4', save = True)

In [112]:
ionization_vis(gas_data, edgeE_data, elec_data, 5,\
                   t, LX, LY, savePath, \
                   'ionization_vis_8_30.mp4', save = True)

In [6]:
np.shape(elec_data[-1])

(28567, 7)

# Set up for recombination

In [18]:
np.shape(Ar2_data[-1])

(124, 7)

In [10]:
import format_VSim_input as fv
infile = simPath +  'final_electrons.txt'
outfile = simPath + 'input_electrons.txt'
fv.format(infile, outfile)

In [11]:
import format_VSim_input as fv
infile = simPath +  'final_ions.txt'
outfile = simPath + 'input_ions.txt'
fv.format(infile, outfile)

In [19]:
import format_VSim_input as fv
infile = simPath + 'final_Ar2.txt'
outfile = simPath + 'input_Ar2.txt'
fv.format(infile, outfile)